In [2]:
using Gen, Statistics, StatsPlots, Memoize, BenchmarkTools, Distributions
import POMDPs, POMDPModels, POMDPModelTools, POMDPGifs

We tried this example and it doesn't work, just returns some random trajectory with no regard to score (or so it seems)

In [15]:
actions = [:up,:down,:left,:right]
grid_world = [
    "V" "#" ;
    "___" "___";

]

grid_world_utilities = Dict(
  "DS" => 1, 
  "DN" => 1, 
  "V" => 3,
  "N" => 2, 
  "___"=> -0.1,
  "#" => -1000000
)
n_rows, n_cols = size(grid_world)
grid_size = (n_cols, n_rows)
function grid_to_utilities_dict(grid_world)
    utilities_dict = Dict()
    for i=1:n_rows
        for j=1:n_cols
            utility = grid_world_utilities[grid_world[n_rows+1 - i,j]]
            utilities_dict[POMDPModels.GWPos(j, i)] = utility
        end
    end
    return utilities_dict
end

function get_terminal_states(grid_world)
    terminal_states = Set()
    for i=1:n_rows
        for j=1:n_cols 
            world_object = grid_world[n_rows+1-i,j]
            if world_object != "___"
                push!(terminal_states,POMDPModels.GWPos(j,i))
            end
        end
    end
    return terminal_states
end

start_state = POMDPModels.GWPos(2, 1)
gw = POMDPModels.SimpleGridWorld(size=grid_size,
                                    rewards=grid_to_utilities_dict(grid_world),
                                    terminate_from = get_terminal_states(grid_world),
                                    tprob = 1,
                                    discount = 1)

function transition(state,action)
    return POMDPModelTools.rand(POMDPs.transition(gw, state, action))
end

function utility(state)
    return gw.rewards[state]
end

function is_terminal_state(state)
    return POMDPs.isterminal(gw, state)
end

function make_agent()
    @gen function act(state, time_left)
        action_index = @trace(uniform_discrete(1,4),:action_index)
        eu = expected_utility(state, actions[action_index], time_left)
        @trace(bernoulli(exp(100 * (eu - 3))), :factor)
        return action_index
    end

    @memoize function run_act(state, time_left)
        action_indices = []
        trace, = generate(act, (state, time_left), choicemap((:factor,1)))
        for i = 1:1000
            trace, = Gen.mh(trace, select(:action_index))
            push!(action_indices, get_retval(trace))
        end
        return action_indices
    end

    @gen function reward(state, action, time_left)
        next_state = transition(state, action)
        action_indices = run_act(state, time_left)
        rand_choice = @trace(uniform_discrete(1, length(action_indices)), :rand_choice)
        next_action_idx = action_indices[rand_choice]
        return expected_utility(next_state, actions[next_action_idx], time_left)
    end

    @memoize function run_reward(state, action, time_left)
        rewards = []
        trace, = generate(reward, (state, action, time_left))
        for i = 1:1000
            trace, = Gen.mh(trace, select(:rand_choice))
            push!(rewards, get_retval(trace))
        end
        return rewards
    end

    @memoize function expected_utility(state, action, time_left)
        u = utility(state)
        next_state = transition(state, action)
        new_time_left = time_left - 1;
        if is_terminal_state(next_state) || new_time_left == 0
            return u
        else
            return u + mean(run_reward(state, action, new_time_left))
        end
    end
    return run_act
end

function simulate(start_state, total_time)
    states = []
    run_act = make_agent()
    next_state = start_state
    while !is_terminal_state(next_state) && total_time > 0
        push!(states, next_state)
        action_indices = run_act(next_state, total_time)
        rand_choice = rand(DiscreteUniform(1, length(action_indices)))
        next_action = action_indices[rand_choice]
        next_state = transition(next_state, actions[next_action])
        total_time -= 1
    end
    return states
end;

In [16]:
trajectory = simulate(start_state, 5)

5-element Vector{Any}:
 [2, 1]
 [2, 1]
 [2, 1]
 [1, 1]
 [1, 2]

When we try it with an infinite even horizon (meaning no time limit) it encounters a stack overflow, which makes no sense since the grid is so small

In [22]:
actions = [:up,:down,:left,:right]
grid_world = [
    "V" "#" ;
    "___" "___";
]

grid_world_utilities = Dict(
  "DS" => 1, 
  "DN" => 1, 
  "V" => 3,
  "N" => 2, 
  "___"=> -0.1,
  "#" => -1000000
)
n_rows, n_cols = size(grid_world)
grid_size = (n_cols, n_rows)
function grid_to_utilities_dict(grid_world)
    utilities_dict = Dict()
    for i=1:n_rows
        for j=1:n_cols
            utility = grid_world_utilities[grid_world[n_rows+1 - i,j]]
            utilities_dict[POMDPModels.GWPos(j, i)] = utility
        end
    end
    return utilities_dict
end

function get_terminal_states(grid_world)
    terminal_states = Set()
    for i=1:n_rows
        for j=1:n_cols 
            world_object = grid_world[n_rows+1-i,j]
            if world_object != "___"
                push!(terminal_states,POMDPModels.GWPos(j,i))
            end
        end
    end
    return terminal_states
end

start_state = POMDPModels.GWPos(2, 1)
gw = POMDPModels.SimpleGridWorld(size=grid_size,
                                    rewards=grid_to_utilities_dict(grid_world),
                                    terminate_from = get_terminal_states(grid_world),
                                    tprob = 1,
                                    discount = 1)

function transition(state,action)
    return POMDPModelTools.rand(POMDPs.transition(gw, state, action))
end

function utility(state)
    return gw.rewards[state]
end

function is_terminal_state(state)
    return POMDPs.isterminal(gw, state)
end

function make_agent()
    @gen function act(state)
        action_index = @trace(uniform_discrete(1,4),:action_index)
        eu = expected_utility(state, actions[action_index])
        @trace(bernoulli(exp(100 * (eu - 3))), :factor)
        return action_index
    end

    @memoize function run_act(state)
        action_indices = []
        trace, = generate(act, (state,), choicemap((:factor,1)))
        for i = 1:1000
            trace, = Gen.mh(trace, select(:action_index))
            push!(action_indices, get_retval(trace))
        end
        return action_indices
    end

    @gen function reward(state, action)
        next_state = transition(state, action)
        action_indices = run_act(state)
        rand_choice = @trace(uniform_discrete(1, length(action_indices)), :rand_choice)
        next_action_idx = action_indices[rand_choice]
        return expected_utility(next_state, actions[next_action_idx])
    end

    @memoize function run_reward(state, action)
        rewards = []
        trace, = generate(reward, (state, action))
        for i = 1:1000
            trace, = Gen.mh(trace, select(:rand_choice))
            push!(rewards, get_retval(trace))
        end
        return rewards
    end

    @memoize function expected_utility(state, action)
        u = utility(state)
        next_state = transition(state, action)
        if is_terminal_state(next_state)
            return u
        else
            return u + mean(run_reward(state, action))
        end
    end
    return run_act
end

function simulate(start_state)
    states = []
    run_act = make_agent()
    next_state = start_state
    while !is_terminal_state(next_state)
        push!(states, next_state)
        action_indices = run_act(next_state)
        rand_choice = rand(DiscreteUniform(1, length(action_indices)))
        next_action = action_indices[rand_choice]
        next_state = transition(next_state, actions[next_action])
    end
    return states
end;

In [23]:
trajectory = simulate(start_state)

LoadError: StackOverflowError: